<a href="https://colab.research.google.com/github/mohanrajmit/MLDS2020/blob/master/Transfer_Learning_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from pathlib import Path
import numpy as np
import joblib
from keras.preprocessing import image
from keras.applications import xception

# Empty lists to hold the images and labels for each each image
x_train = []
y_train = []

Using TensorFlow backend.


In [0]:
!git clone https://github.com/mohanrajmit/Transfer-Learning.git

Cloning into 'Transfer-Learning'...
remote: Enumerating objects: 2214, done.
remote: Counting objects: 100% (2214/2214), done.
remote: Compressing objects: 100% (2212/2212), done.
remote: Total 2214 (delta 1), reused 2207 (delta 0), pack-reused 0
Receiving objects: 100% (2214/2214), 13.85 MiB | 34.60 MiB/s, done.
Resolving deltas: 100% (1/1), done.


In [0]:
!ls

sample_data  Transfer-Learning


In [0]:
# Load the training data set by looping over every image file
for image_file in Path("Transfer-Learning/training_dataset").glob("**/*.png"):

    # Load the current image file
    image_data = image.load_img(image_file, target_size=(73, 73))

    # Convert the loaded image file to a numpy array
    image_array = image.img_to_array(image_data)

    # Add the current image to our list of training images
    x_train.append(image_array)

    # Add a label for this image. If it was a not_bird image, label it 0. If it was a bird, label it 1.
    if "not_bird" in image_file.stem:
        y_train.append(0)
    else:
        y_train.append(1)

In [0]:
# Convert the list of separate images into a single 4D numpy array. This is what Keras expects.
x_train = np.array(x_train)

# Normalize image data to 0-to-1 range
x_train = xception.preprocess_input(x_train)

# Load the pre-trained neural network to use as a feature extractor
feature_extractor = xception.Xception(weights='imagenet', include_top=False, input_shape=(73, 73, 3))

# Extract features for each image (all in one pass)
features_x = feature_extractor.predict(x_train)

# Save the array of extracted features to a file
joblib.dump(features_x, "x_train.dat")

# Save the matching array of expected values to a file
joblib.dump(y_train, "y_train.dat")


83689472/83683744 [==============================] - 3s 0us/step


['y_train.dat']

In [0]:
# Load data set of extracted features
x_train = joblib.load("x_train.dat")
y_train = joblib.load("y_train.dat")


In [0]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten

In [0]:
# Create a model and add layers
model = Sequential()

# Add layers to our model
model.add(Flatten(input_shape=x_train.shape[1:]))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

# Compile the model
model.compile(
    loss='binary_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)


In [0]:
# Train the model
model.fit(
    x_train,
    y_train,
    validation_split=0.05,
    epochs=10,
    shuffle=True,
    verbose=2
)

# Save the trained model to a file so we can use it to make predictions later
model.save("bird_feature_classifier_model.h5")

Train on 1900 samples, validate on 100 samples
Epoch 1/10
 - 2s - loss: 0.6775 - acc: 0.7495 - val_loss: 0.2600 - val_acc: 0.9000
Epoch 2/10
 - 0s - loss: 0.3672 - acc: 0.8321 - val_loss: 0.4455 - val_acc: 0.8100
Epoch 3/10
 - 0s - loss: 0.3312 - acc: 0.8574 - val_loss: 0.4256 - val_acc: 0.8200
Epoch 4/10
 - 0s - loss: 0.3004 - acc: 0.8611 - val_loss: 0.2432 - val_acc: 0.9300
Epoch 5/10
 - 0s - loss: 0.2968 - acc: 0.8658 - val_loss: 0.4203 - val_acc: 0.8300
Epoch 6/10
 - 0s - loss: 0.2333 - acc: 0.8895 - val_loss: 0.3555 - val_acc: 0.8900
Epoch 7/10
 - 0s - loss: 0.2101 - acc: 0.9053 - val_loss: 0.3109 - val_acc: 0.9000
Epoch 8/10
 - 1s - loss: 0.1969 - acc: 0.9132 - val_loss: 0.3250 - val_acc: 0.9200
Epoch 9/10
 - 0s - loss: 0.1729 - acc: 0.9184 - val_loss: 0.3711 - val_acc: 0.8900
Epoch 10/10
 - 1s - loss: 0.1752 - acc: 0.9284 - val_loss: 0.6389 - val_acc: 0.8000


In [0]:
from keras.models import load_model
from sklearn.metrics import confusion_matrix, classification_report

In [0]:
# Empty lists to hold the images and labels for each each image
x_test = []
y_test = []

# Load the test data set by looping over every image file
for image_file in Path("Transfer-Learning/test_dataset").glob("**/*.png"):

    # Load the current image file
    image_data = image.load_img(image_file, target_size=(73, 73))

    # Convert the loaded image file to a numpy array
    image_array = image.img_to_array(image_data)

    # Add the current image to our list of test images
    x_test.append(image_array)

    # Add an expected label for this image. If it was a not_bird image, label it 0. If it was a bird, label it 1.
    if "not_bird" in image_file.stem:
        y_test.append(0)
    else:
        y_test.append(1)

# Convert the list of test images to a numpy array
x_test = np.array(x_test)

# Normalize test data set to 0-to-1 range
x_test = xception.preprocess_input(x_test)

In [0]:
# Load our trained classifier model
model = load_model("bird_feature_classifier_model.h5")

# Extract features for each image (all in one pass)
features_x = feature_extractor.predict(x_test)

# Given the extracted features, make a final prediction using our own model
predictions = model.predict(features_x)

# If the model is more than 50% sure the object is a bird, call it a bird.
# Otherwise, call it "not a bird".
predictions = predictions > 0.5

# Calculate how many mis-classifications the model makes
tn, fp, fn, tp = confusion_matrix(y_test, predictions).ravel()
print(f"True Positives: {tp}")
print(f"True Negatives: {tn}")
print(f"False Positives: {fp}")
print(f"False Negatives: {fn}")

# Calculate Precision and Recall for each class
report = classification_report(y_test, predictions)
print(report)

True Positives: 88
True Negatives: 75
False Positives: 25
False Negatives: 12
              precision    recall  f1-score   support

           0       0.86      0.75      0.80       100
           1       0.78      0.88      0.83       100

   micro avg       0.81      0.81      0.81       200
   macro avg       0.82      0.81      0.81       200
weighted avg       0.82      0.81      0.81       200

